In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
from sqlalchemy import create_engine

# import dateutil.parser
from dateutil.parser import parse
# from dateutil.parser import parse
import re
# from ast import literal_eval
import ast

In [30]:
"""Import and load the data into a dataframe."""
data = pd.read_csv("cleaned_best_books_ever.csv")
df = pd.DataFrame(data)

In [21]:
# Create a new DataFrame with bookId and exploded genres
genres_df = df[['bookId', 'genres']].copy()
genres_df['genres'] = genres_df['genres'].str.split(',')
exploded_genres_df = genres_df.explode('genres').reset_index(drop=True)
# exploded_genres_df = genres_df.explode('genres').replace("[]'", '')
# exploded_genres_df['genres'] = exploded_genres_df['genres'].replace('\W+', '')
exploded_genres_df['genres'] = exploded_genres_df['genres'].replace(r'[^0-9a-zA-Z ]', '', regex=True)#.replace("'", '')
# exploded_genres_df.head(20)

# Rename the columns for clarity
exploded_genres_df.columns = ['bookId', 'genreName']
exploded_genres_df['genreName'] = exploded_genres_df['genreName'].str.strip()  # Removing any leading or trailing whitespace

exploded_genres_df.head(20)

982

In [ ]:
def analyze_categorical_column(dataframe, column_name):
    
    # Check if the column exists in the DataFrame
    if column_name not in dataframe.columns:
        return f"The column '{column_name}' does not exist in the DataFrame."
    
    # 1. Check for missing values
    analysis_results["missing_values"] = dataframe[column_name].isnull().sum()

    # 2. Check for uniqueness
    analysis_results["unique_values"] = dataframe[column_name].nunique()
    analysis_results["total_values"] = dataframe[column_name].count()
    analysis_results["duplicated_values"] = dataframe[column_name].duplicated().sum()

    # 3. Check for values with leading or trailing whitespace
    analysis_results["values_with_whitespace"] = dataframe[column_name].str.strip().ne(dataframe[column_name]).sum()

    # 4. Check for values with unusual characters (non-printable ASCII)
    analysis_results["values_with_unusual_chars"] = dataframe[column_name].apply(lambda x: any(ord(char) < 32 or ord(char) > 126 for char in str(x))).sum()

    # 5. Get the distribution of value lengths
    value_lengths = dataframe[column_name].str.len()
    analysis_results["shortest_value_length"] = value_lengths.min()
    analysis_results["longest_value_length"] = value_lengths.max()
    analysis_results["average_value_length"] = value_lengths.mean()
    
    shortest_value = dataframe[dataframe[column_name].str.len() == analysis_results['shortest_value_length']]
    longest_value = dataframe[dataframe[column_name].str.len() == analysis_results['longest_value_length']]
    
    return analysis_results, shortest_value, longest_value

In [32]:
# Create a new DataFrame with bookId and exploded genres
genres_df = df[['bookId', 'genres']].copy()
genres_df['genres'] = genres_df['genres'].str.split(',')
genres_df = genres_df.explode('genres').reset_index(drop=True).replace(r'[^0-9a-zA-Z ]', '', regex=True)

# Rename the columns for clarity
genres_df.columns = ['bookId', 'genreName']
genres_df['genreName'] = genres_df['genreName'].str.strip()  # Removing any leading or trailing whitespace

exploded_genres_df.head(20)

,bookId,genreName
0,2767052,young adult
1,2767052,fiction
2,2767052,teen
3,2767052,fantasy
4,2767052,science fiction
5,2767052,romance
6,2767052,post apocalyptic
7,2767052,action
8,2767052,dystopia
9,2767052,adventure


In [42]:
isbn = df[['isbn', 'bookId']].drop_duplicates()

isbn.head()


,isbn,bookId
0,9780439023481,2767052
1,9780439358071,2
2,9999999999999,2657
3,9999999999999,1885
4,9780316015844,41865


In [41]:
df[['isbn', 'bookId']].nunique()

isbn      43377
bookId    52424
dtype: int64